In [1]:
import numpy as np
import scipy.stats as stats
from matplotlib import pyplot as plt
np.set_printoptions(precision = 3, suppress = True)

In [2]:
# Load data
flight_data = np.genfromtxt('./files/nycflights.csv',delimiter = ",", skip_header=1)

In [3]:
print(flight_data[0])

[   1. 2013.    1.    1.  517.    2.  830.   11.   nan   nan 1545.   nan
   nan  227. 1400.    5.   17.   nan]


In [4]:
print(flight_data.shape)

(336776, 18)


In [5]:
# Get Input data for model
dep_delay = np.array([row[5] for row in flight_data])
distance = np.array([row[-3] for row in flight_data])
arr_delay = np.array([row[7] for row in flight_data])
X = np.array([dep_delay,distance,arr_delay]).T
print(X)

[[ 2.  5. 11.]
 [ 4.  5. 20.]
 [ 2.  5. 33.]
 ...
 [nan nan nan]
 [nan nan nan]
 [nan nan nan]]


# Cleaning data - remove NaN

In [11]:
# Có 8255 giá trị NaN trong cột dep_delay
np.sum(np.isnan(dep_delay))

8255

In [10]:
# Có 9430 giá trị NaN trong cột arr_delay
np.sum(np.isnan(arr_delay))

9430

In [12]:
#index của các giá trị NaN trong tập dep_delay
dep_delay_index = np.argwhere(np.isnan(dep_delay)).squeeze()
dep_delay_index

array([   838,    839,    840, ..., 336773, 336774, 336775], dtype=int64)

In [13]:
#index của các giá trị NaN trong tập arr_delay
arr_delay_index = np.argwhere(np.isnan(arr_delay)).squeeze()
arr_delay_index

array([   471,    477,    615, ..., 336773, 336774, 336775], dtype=int64)

In [24]:
index_nan = list(dep_delay_index) + list(arr_delay_index)
print(np.sort(index_nan).shape)
index_nan = np.unique(np.sort(index_nan))
print(np.unique(np.sort(index_nan)))

(17685,)
[   471    477    615 ... 336773 336774 336775]


In [26]:
index_nan.shape

(9430,)

In [197]:
# Remove NaN
cleaned_X = X[~np.isnan(arr_delay)]
cleaned_X

array([[  2.,   5.,  11.],
       [  4.,   5.,  20.],
       [  2.,   5.,  33.],
       ...,
       [ -5.,  22., -16.],
       [ 12.,  23.,   1.],
       [-10.,  23., -25.]])

In [198]:
data_input = cleaned_X[:,0:2]
data_input

array([[  2.,   5.],
       [  4.,   5.],
       [  2.,   5.],
       ...,
       [ -5.,  22.],
       [ 12.,  23.],
       [-10.,  23.]])

In [199]:
data_output = cleaned_X[:,2]
data_output

array([ 11.,  20.,  33., ..., -16.,   1., -25.])

# # Chia dữ liệu tập train và tập test

In [200]:
np.random.seed(5)
random_index = np.arange(data_output.size)
np.random.shuffle(random_index)
print(random_index)
print(random_index.shape)

[ 10161 171105 215210 ...  20463  18638 297827]
(327346,)


In [201]:
N_TRAIN = 300000
train_index = random_index[:N_TRAIN]
test_index = random_index[N_TRAIN:]
print(train_index.shape)
print(test_index.shape)

(300000,)
(27346,)


In [202]:
x_train, y_train = data_input[train_index], data_output[train_index]
print(x_train)
print(y_train)

x_test, y_test = data_input[test_index], data_output[test_index]
print(x_test)
print(y_test)

[[ -4.  16.]
 [  3.  10.]
 [128.  21.]
 ...
 [ -2.  10.]
 [184.  23.]
 [ -5.  10.]]
[-21.   6.  89. ... -24. 158. -19.]
[[-7.  8.]
 [ 7. 15.]
 [ 2. 20.]
 ...
 [19. 18.]
 [-4. 17.]
 [-5.  6.]]
[-11.  21. -22. ... -14.  -5. -21.]


##  Tạo mô hình Linear Regression
Sử dụng lớp `LinearRegression` của thư viện `sklearn`

In [203]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()

In [204]:
#Train model
model.fit(x_train,y_train)

LinearRegression()

In [205]:
r2 = model.score(x_train,y_train)
r2

0.8368409538359806

In [206]:
slope_a = model.coef_
slope_a

array([ 1.021, -0.049])

In [207]:
intercept = model.intercept_
intercept

-5.281095685038107

In [214]:
arr_delay_10 = model.predict([x_train[0]])[0]
print(arr_delay_10)
print(y_train[10])

-10.143984974788367
-15.0


# Đánh giá trên tập test

In [215]:
q2  = model.score(x_test, y_test)
print(q2)

0.8374348661818591


In [216]:
y_test_hat = model.predict(x_test)
print(y_test_hat)
print(y_test_hat - y_test)

[-12.816   1.133  -4.214 ...  13.237 -10.193 -10.677]
[ -1.816 -19.867  17.786 ...  27.237  -5.193  10.323]


In [221]:
arr_delay_test1 = model.predict([x_test[100]]).squeeze()
print(arr_delay_test1)

-10.774912720888459


In [222]:
y_test[100]

-26.0